In [ ]:
!pip install transformers datasets -qq

In [2]:
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
from tqdm import tqdm

In [3]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [4]:
ds = load_dataset("shpotes/tfcol", split="validation")

Using custom data configuration default
Reusing dataset tf_col (/root/.cache/huggingface/datasets/shpotes___tf_col)/default/1.0.0/0c616218d5e0a194334e0ed0adacd86ab9b315ec6b03a8b388dece024753def2)


In [5]:
def int2str(x):
  _int2str = ds.features["labels"].feature.int2str
  if isinstance(x, int):
    return _int2str(x)
  elif hasattr(x, '__iter__'):
    return [_int2str(i) for i in x]
  raise TypeError

def batch(iterable, n=1):
  l = len(iterable)
  num = range(l)
  for ndx in range(0, l, n):
    yield num[ndx:min(ndx + n, l)], iterable[ndx:min(ndx + n, l)]

In [6]:
images = [Image.open(img).convert("RGB") for img in ds["image"]]

prompt_seeds = [
    "clothing store",
    "liquor store",
    "barber shop",
    "electronic store",
    "coffee store",
    "furniture store",
    "hot dog cart", # puesto movil 
    "ERROR OSJDFADOIAJSOIDJAMS", # electrodomesticos
    "butcher shop",
    "bar",
    "pet shop",
    "store",
    "pharmacy",
    "sport store",
    "car shop",
    "shoe shop",
    "supermarket",
    "hotel"
]

prompts = [f"an image of a {seed}" for seed in prompt_seeds]

model = model.cuda()

In [20]:
score = [0 for _ in range(len(prompts))]

In [31]:
for image_idx, image_batch in tqdm(batch(images, 8)):
  input = processor(
      text=prompts, 
      images=image_batch,
      return_tensors="pt", 
      padding=True,
  )
  
  input = {k: v.cuda() for k, v in input.items()}

  outputs = model(**input)
  logits_per_image = outputs.logits_per_image
  probs = logits_per_image.softmax(dim=1)

  for img_idx, prob_idx in zip(image_idx, range(8)):
    topk = set(torch.topk(probs[prob_idx], 5).indices.cpu().tolist())
    labels = set(ds["labels"][img_idx])


    for i in (topk & labels):
      score[i] += 1

83it [00:25,  3.21it/s]


In [32]:
final_score = [local / len(ds) for local in score]

In [33]:
final_score

[0.11550151975683891,
 0.015197568389057751,
 0.04711246200607903,
 0.04559270516717325,
 0.0060790273556231,
 0.04711246200607903,
 0.00303951367781155,
 0.0,
 0.019756838905775075,
 0.00303951367781155,
 0.025835866261398176,
 0.022796352583586626,
 0.00303951367781155,
 0.0060790273556231,
 0.004559270516717325,
 0.0,
 0.0121580547112462,
 0.0]